# Importing files and python essencial library
This notebook contains all the precedure since data preparing, processing, ArcPy generation and data analysis

In [1]:
# This is the main Libraries applied in our research
import pandas as pd
import numpy as np
import os
os.listdir('')

In [9]:
# Defining the path of files generated by the script
from google.colab import drive
drive.mount('/content/drive/')
pathRaw = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/RawFile/'
outPutEstocastic = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/EstocasticSamples/EstocasticSamplesTest1/'
outPutDeterministic = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/DeterministicSample/deterministicSamples1/'
outPutTestfile = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/testSample/TestSamples1/'
rawFileName = 'baseFile.csv'
baseFile =  pd.read_csv('/content/drive/MyDrive/TCC/baseFile/baseFile.csv', decimal = ',')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
# executing the sum of each temporal window;
# pluvio trimester
baseFile['pluvio_trimester']= baseFile.pluvio_jan + baseFile.pluvio_fev + baseFile.pluvio_mar
baseFile['value_trimester'] = baseFile.value_fev + baseFile.value_jan + baseFile['value_mar ']

## Ploting a map
* Map of the Rain Gauges Analyzed

In [25]:
import folium
m = folium.Map(location=[-23.693635, -46.583919], zoom_start=10, tiles='Stamen Terrain' )
# Iterate over the DataFrame rows and add markers
icon_size_css = 'font-size: 15px; font-weight: bold;'
for index, row in baseFile.iterrows():
    marker = folium.Marker(
        location=[row['ycoord'], row['xcoord']],
        icon=folium.DivIcon(html=f'<div style="{icon_size_css}">{row["prefixo"]}</div>')
    )
    marker.add_to(m)



    folium.Marker(
        location=[row['ycoord'], row['xcoord']],
        popup=row['prefixo']  # Add Location Name as a popup
    ).add_to(m)
# Display the map
m.save('map_with_dataframe.html')  # Save the map to an HTML file
m

# Selecting testing pluviometer  
* Look that 1000550 is really close to 1000430, this can affect the results
* First test we considered: (157), (1000550) e (1000430)
* In the second test we considered 629, 1000550, 1000490, 1000380;

In [37]:
baseFile.columns

Index(['prefixo', 'pluvio_jan', 'pluvio_fev', 'pluvio_mar', 'value_jan',
       'value_fev', 'value_mar ', 'ycoord', 'xcoord', 'pluvio_trimester',
       'value_trimester'],
      dtype='object')

In [38]:
baseFile.prefixo.astype(str)

0     1000370
1     1000380
2     1000400
3     1000410
4     1000430
5     1000490
6     1000500
7     1000510
8     1000817
9     1000868
10    1000938
11    1000958
12        143
13        157
14        279
15        280
16        283
17        413
18        511
19        514
20        528
21        563
22        629
23    1000350
24    1000550
Name: prefixo, dtype: object

In [39]:
testRainGauges = ['629', '1000550', '1000490', '1000380']
testFile = baseFile[baseFile['prefixo'].astype(str).isin(testRainGauges)]

In [40]:
testFile

,prefixo,pluvio_jan,pluvio_fev,pluvio_mar,value_jan,value_fev,value_mar,ycoord,xcoord,pluvio_trimester,value_trimester
1,1000380,361.4,447.8,291.6,1140.408,1998.051,2133.364,-23.699826,-46.588495,1100.8,5271.823
5,1000490,140.6,138.0,0.0,1074.149,241.453,0.000,-23.662847,-46.468214,278.6,1315.602
22,629,242.0,418.4,214.8,4261.881,500.644,1664.190,-23.595779,-46.601021,875.2,6426.715
24,1000550,202.4,91.8,63.6,1989.832,259.935,1324.337,-23.664317,-46.587103,357.8,3574.104


# Selecting training files for deterministic model

In [41]:
deterministicRainGauges = baseFile[~baseFile['prefixo'].astype(str).isin(testRainGauges)]

In [42]:
print('\n Number of Rain Gauges: ', len(baseFile), '\n Number of testing Rain Gauges: ', len(testFile),
     '\n Number of Rain Gauges in Deterministic Model: ', len(deterministicRainGauges))


 Number of Rain Gauges:  25 
 Number of testing Rain Gauges:  4 
 Number of Rain Gauges in Deterministic Model:  21


# Selecting training files for stocastic model
* Selecting combination of 14 by 14 rain gauges with no repetition;
* Generating 30 tables; each table is a combination of rain gauges;

In [43]:
rainGauges = deterministicRainGauges

In [44]:
import pandas as pd
from itertools import combinations
import random

# Assuming you have the 'rainGauges' DataFrame

# Number of rows in each sub-DataFrame

chunk_size = 14

# Generate all possible combinations of row indices
indices_combinations = list(combinations(rainGauges.index, chunk_size))

# Check if there are at least 30 combinations available
if len(indices_combinations) < 30:
    raise ValueError("There are not enough combinations to form 30 sub-DataFrames.")

# Randomly select 30 unique combinations
selected_indices = random.sample(indices_combinations, 30)

# Create a list to store the sub-DataFrames
sub_dataframes = []

# Loop through the selected indices and create sub-DataFrames
for indices in selected_indices:
    sub_df = rainGauges.loc[list(indices)].copy()
    sub_dataframes.append(sub_df)

# Now, 'sub_dataframes' contains 30 DataFrames, each with 14 unique rows from 'rainGauges'


In [45]:
sub_dataframes[0]

,prefixo,pluvio_jan,pluvio_fev,pluvio_mar,value_jan,value_fev,value_mar,ycoord,xcoord,pluvio_trimester,value_trimester
3,1000410,262.00,381.60,257.6,1852.159,1125.180,5472.208,-23.717416,-46.602312,901.2,8449.547
6,1000500,167.80,417.60,206.6,2947.102,687.226,2636.083,-23.597551,-46.541079,792.0,6270.411
7,1000510,171.25,381.25,239.5,1640.776,1046.696,3513.631,-23.650634,-46.440583,792.0,6201.103
8,1000817,155.60,314.80,156.6,1062.512,1521.906,1337.920,-23.598270,-46.544513,627.0,3922.338
9,1000868,245.60,425.00,254.0,2658.145,3068.642,1894.363,-23.587240,-46.633968,924.6,7621.150
10,1000938,235.50,543.00,312.0,1902.741,1349.956,564.786,-23.650069,-46.644951,1090.5,3817.483
11,1000958,188.00,366.00,174.0,4043.093,256.038,2717.517,-23.596712,-46.584471,728.0,7016.648
14,279,247.00,300.80,311.0,4846.082,2057.235,2482.261,-23.716175,-46.601744,858.8,9385.578
15,280,169.00,404.60,300.0,1896.741,432.858,1241.154,-23.612190,-46.530731,873.6,3570.753
17,413,276.00,400.40,272.4,2480.233,1082.666,3640.286,-23.542365,-46.629651,948.8,7203.185


In [46]:
treated_subDataframes = []
for dataframe in sub_dataframes:
    treated_subDataframes.append(dataframe[['prefixo', 'pluvio_trimester', 'ycoord', 'xcoord']])

# Exporting dataframes

### Exporting Stocastic Samples

In [47]:
number_of_samples = 0
for dataframe in treated_subDataframes:
    dataframe.to_csv(outPutEstocastic + 'EstocasticSample_' +str(number_of_samples) +'.csv', index = None)
    number_of_samples+=1

OSError: Cannot save file into a non-existent directory: 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/EstocasticSamples/EstocasticSamplesTest1'

### Exporting Deterministic Samples

In [ ]:
deterministicRainGauges[['prefixo', 'pluvio_trimester', 'ycoord', 'xcoord']].to_csv(
    outPutDeterministic + 'DeterministicSamples2' + '.csv', index = None)

### Exporting Testing Samples

In [ ]:
testFile.to_csv(outPutTestfile + "TestSamples_2"+'.csv', index = None)

# (ARCGIS SCRIPT)
* This part of the code contain the data processed in software AricGIS Pro with ArcPy API;


In [ ]:
outputPath = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut/'
outputFileNames = 'ValueExtracted'

## Importing all csv files as points

In [ ]:
import arcpy
import os
from arcpy import env
# Set up the workspace and environment
env.workspace = r'C:\Users\vitor.yuichi\Desktop\Pasta Vitor\Univ\modeloEstocasticoDeterministico\EstocasticSamples\EstocasticSamplesTest1\ ' # Set the path to your workspace directory
csv_files =arcpy.ListFiles("*.csv")
# Loop through all the CSV files in the workspace
for csv_file in csv_files:
    # Create a name for the output point feature class based on the CSV file name
    output_name = arcpy.ValidateTableName(csv_file.replace(".csv", ""), env.workspace)
    # Make XY Event Layer
    in_table = csv_file
    x_coords = "xcoord"  # Replace with the actual X field name in the CSV
    y_coords = "ycoord"  # Replace with the actual Y field name in the CSV
    spatial_reference = arcpy.SpatialReference(4674)  # Replace with the appropriate spatial reference co
    arcpy.MakeXYEventLayer_management(in_table, x_coords, y_coords, output_name, spatial_reference)

## Generating all Raster Values

In [ ]:
import arcpy
from arcpy.sa import *

# Set the workspace to the current project
arcpy.env.workspace = arcpy.mp.ArcGISProject("CURRENT").defaultGeodatabase

# Define the directory to save the output rasters
output_directory = r"C:\Users\vitor.yuichi\Desktop\Pasta Vitor\Univ\modeloEstocasticoDeterministico\rasterFile\RasterFilesTest1"

# Get a list of all active map layers in the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")
active_layers = [lyr for lyr in aprx.listMaps()[0].listLayers() if lyr.isFeatureLayer]

# List to hold the rasters created from IDW interpolation of each layer
rasters = []

# Perform IDW interpolation for each active layer and store the resulting raster in the 'rasters' list
for layer in active_layers:
    output_raster = f"{layer.name}_IDW"
    out_idw = Idw(layer, "pluvio_trimester", 0.001, 1.5)

    # Save the raster to the specified output directory
    output_path = os.path.join(output_directory, f"{output_raster}.tif")
    out_idw.save(output_path)

    rasters.append(output_path)
    print(f"IDW interpolation for {layer.name} completed.")

## Extracting value from each test sample
* This script extract the value of the selected rain gauges for test for each test sample;

#### Colecting the name of all current active TIF files;

In [ ]:
import arcpy
list_of_tifs = []
# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Get a list of all active layers in the current project
active_layers = list(aprx.listMaps()[0].listLayers())
for layer in active_layers:
    list_of_tifs.append(layer.name)

# arcpy.env.workspace = "C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut"

In [ ]:
list_of_tifs[0][:-4]

#### For each test sample with extracted value, we labeled with the names of TIF file generated

In [ ]:
import arcpy

# Set the workspace if needed
# arcpy.env.workspace = "your_workspace_path"

# Define the name of the point layer

point_layer = "TestSamples_2_"

# Define the name of the active raster layer
raster_layer_list = list_of_tifs

for raster_layer, name in zip(raster_layer_list, [var[:-4] for var in list_of_tifs]) :
# Define the output feature class for the Extract Values to Points tool
    output_fc = name

# Execute the Extract Values to Points tool
    arcpy.sa.ExtractValuesToPoints(point_layer, raster_layer, output_fc, "INTERPOLATE", "VALUE_ONLY")


print("Values extracted successfully to points.")


In [ ]:
import arcpy
import pandas as pd

# Define the path to your feature class or layer
# input_fc = "output_points_with_values"

# Define the path for the output Excel file
for name in [var[:-4] for var in list_of_tifs]:

# Define the path to your feature class or layer
    input_fc = name
    output_excel = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut/OutPutForTest1/' + name + '.xlsx'
# Use the arcpy.da.SearchCursor to extract the attribute table data
    field_names = [field.name for field in arcpy.ListFields(input_fc)]
    data = [row for row in arcpy.da.SearchCursor(input_fc, field_names)]

# Convert the attribute table data to a Pandas DataFrame
    df = pd.DataFrame(data, columns=field_names)

# Save the DataFrame to an Excel file
    df.to_excel(output_excel, index=False)

print("Attribute table saved as Excel successfully.")


## Data Analysis of the Results

## Importing data to analyze and concating the multiple generated files with value extracted for each testing point

In [48]:
import os
import pandas as pd
import numpy as np
os.listdir('C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut/OutPutForTest1')
outputpath = 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut/OutPutForTest1/'

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/vitor.yuichi/Desktop/Pasta Vitor/Univ/modeloEstocasticoDeterministico/OutPut/OutPutForTest1'

In [49]:
list_of_dataframes = []
for nameFile in os.listdir(outputpath)[:-1]:
    list_of_dataframes.append(pd.read_excel(outputpath + nameFile))

NameError: name 'outputpath' is not defined

In [50]:
os.listdir(outputpath)[:-1]

NameError: name 'outputpath' is not defined

In [ ]:
 valores = pd.read_excel(outputpath + 'Extract_TestSam1_ExportTable_TableToExcel.xlsx')

In [ ]:
concatDfs = pd.concat(list_of_dataframes)
concatDfs = concatDfs.reset_index()

In [ ]:
concatDfs

In [ ]:
data = pd.DataFrame({'1000550': concatDfs[concatDfs['prefixo'] == 1000550]['RASTERVALU'].tolist(),
             '629': concatDfs[concatDfs['prefixo'] == 629]['RASTERVALU'].tolist(),
             '1000490': concatDfs[concatDfs['prefixo'] == 1000490]['RASTERVALU'].tolist(),
             '1000380': concatDfs[concatDfs['prefixo'] == 1000380]['RASTERVALU'].tolist()
             })
data1 = data.iloc[1:,:]
data.iloc[:1,:]

In [ ]:
data

## Plot and discussion
* Modelo Deteminístico: é o raster formado pelos 21 pluviômetros com excessão dos pluviômetros de teste;
* Modelo estocástico são 30 interpolações de 14 pluviômetro permutando entre si sem repetição;
* Observe que as interpolações oriundas do modelo determinístico foi razoável somente no pluviômetro 1000430;
* temos um Desvio grande por parte dos modelos determinístico;
* No modelo estocástico temos somente o pluviômetro 1000550 dentro do intervalor do boxplot gerado; os outros pluviômetro obtiveram um grande desvio;
* Como há grande proximidade entre o pluviômetro 1000550 e 100430 houve influência dos do 1000550 em 1000430 tornando o desvio maior na área;
* o pluviômetro 157 está na extremidade e não contém alguns valores pois o Raster gerado pelas combinações não alcançou o pluviômetro;
* de maneira geral nenhum dos modelos foi razoável para aproximar-se do valor teste nos três pluviômetros;

In [ ]:
valores

In [ ]:
valores

In [ ]:
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.graph_objs as go
import pandas as pd

# Sample data (you can replace this with your own data)


# Create a list of box traces for each column
box_traces = []
for column in data.columns:
    box_trace = go.Box(y=data1[column], name=column)
    box_traces.append(box_trace)

# Adicionar um marcador no primeiro boxplot
valor_do_simbolo = 875
valor_do_simbolo1= 357
valor_do_simbolo2 = 278
valor_do_simbolo3 = 1100.8
valor_do_simbolo4 = 816
valor_do_simbolo5 = 857
valor_do_simbolo6 = 843
valor_do_simbolo7 = 868

# Criar uma trace de marcador
simbolo_trace = go.Scatter(x=['629'], y=[valor_do_simbolo],
                          mode='markers', name="629 point value",
                          marker=dict(color='red', symbol='x', size=10))

simbolo_trace1 = go.Scatter(x=['1000550'], y=[valor_do_simbolo1],
                          mode='markers', name="1000550 point value",
                          marker=dict(color='blue', symbol='x', size=10))

simbolo_trace2 = go.Scatter(x=['1000490'], y=[valor_do_simbolo2],
                          mode='markers', name="1000490 point value",
                          marker=dict(color='green', symbol='x', size=10))

simbolo_trace3 = go.Scatter(x=['1000380'], y=[valor_do_simbolo3],
                          mode='markers', name="1000380 point value",
                          marker=dict(color='Pink', symbol='x', size=10))


simbolo_trace4 = go.Scatter(x=['1000490'], y=[valor_do_simbolo4],
                          mode='markers', name="1000490 determinist value",
                          marker=dict(color='green', symbol='diamond', size=10, ))

simbolo_trace5 = go.Scatter(x=['629'], y=[valor_do_simbolo5],
                          mode='markers', name="629 determinist value",
                          marker=dict(color='red', symbol='diamond', size=10))
simbolo_trace6 = go.Scatter(x=['1000550'], y=[valor_do_simbolo6],
                          mode='markers', name="1000550 determinist value",
                          marker=dict(color='blue', symbol='diamond', size=10))

simbolo_trace7 = go.Scatter(x=['1000380'], y=[valor_do_simbolo7],
                          mode='markers', name="1000380 determinist value",
                          marker=dict(color='pink', symbol='diamond', size=10))
# Create the layout
layout = go.Layout(
    title='Stocastic Distribution',
    xaxis=dict(title='Prefixes'),
    yaxis=dict(title='Extracted Values')
)

# Create the figure and plot it
fig = go.Figure(data=box_traces + [simbolo_trace] +  [simbolo_trace1] +  [simbolo_trace2] +
                [simbolo_trace3] + [simbolo_trace4] + [simbolo_trace5]+ [simbolo_trace6]+
                [simbolo_trace7], layout=layout)
fig.show()
